In [1]:
import mindspore
from mindspore.dataset import transforms

from mindnlp.engine import Trainer

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the defa

In [2]:
from mindnlp.dataset import load_dataset

imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [3]:
imdb_train.get_dataset_size()

25000

In [4]:
from mindnlp.transformers import AutoTokenizer
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

test_tokenized = tokenizer('hello')
test_tokenized.keys()

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [5]:
def process_dataset(dataset, tokenizer, max_seq_len=256, batch_size=32, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['token_type_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'token_type_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'token_type_ids': (None, 0),
                                                             'attention_mask': (None, 0)})

    return dataset

In [6]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(imdb_train, tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, tokenizer)
dataset_test = process_dataset(imdb_test, tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[32, 256], dtype=Int64, value=
 [[ 101, 1398,  178 ...    0,    0,    0],
  [ 101, 1188, 1437 ...    0,    0,    0],
  [ 101, 5145, 2568 ... 1116, 1132,  102],
  ...
  [ 101, 3517,  125 ...    0,    0,    0],
  [ 101,  146, 1541 ...    0,    0,    0],
  [ 101, 1188, 1110 ...    0,    0,    0]]),
 Tensor(shape=[32, 256], dtype=Int64, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  ...
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 Tensor(shape=[32, 256], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 1, 1, 1],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[32], dtype=Int32, value= [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 
  0, 0, 1, 0, 1, 0, 1, 1])]

In [9]:
from mindnlp.transformers import AutoModelForSequenceClassification

# set bert config and define parameters for training
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [10]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="bert_imdb_finetune",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=3.0,
    learning_rate=2e-5
)

In [11]:
from mindnlp import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

  0%|                                                                                                | 0/1641 …

{'loss': 0.3206, 'learning_rate': 2e-05, 'epoch': 1.0}


  0%|                                                                                                 | 0/235 …

{'eval_loss': 0.2645550072193146, 'eval_accuracy': 0.89, 'eval_runtime': 21.6942, 'eval_samples_per_second': 10.832, 'eval_steps_per_second': 1.383, 'epoch': 1.0}
{'loss': 0.1771, 'learning_rate': 2e-05, 'epoch': 2.0}


  0%|                                                                                                 | 0/235 …

{'eval_loss': 0.28401243686676025, 'eval_accuracy': 0.9021333333333333, 'eval_runtime': 21.4019, 'eval_samples_per_second': 10.98, 'eval_steps_per_second': 1.402, 'epoch': 2.0}
{'loss': 0.1088, 'learning_rate': 2e-05, 'epoch': 3.0}


  0%|                                                                                                 | 0/235 …

{'eval_loss': 0.3973828852176666, 'eval_accuracy': 0.9024, 'eval_runtime': 21.3734, 'eval_samples_per_second': 10.995, 'eval_steps_per_second': 1.404, 'epoch': 3.0}
{'train_runtime': 882.004, 'train_samples_per_second': 59.537, 'train_steps_per_second': 1.861, 'train_loss': 0.20214975555810458, 'epoch': 3.0}


TrainOutput(global_step=1641, training_loss=0.20214975555810458, metrics={'train_runtime': 882.004, 'train_samples_per_second': 59.537, 'train_steps_per_second': 1.861, 'train_loss': 0.20214975555810458, 'epoch': 3.0})

In [14]:
trainer.evaluate(dataset_test)

  0%|                                                                                                 | 0/782 …

{'eval_loss': 0.25377440452575684,
 'eval_accuracy': 0.89504,
 'eval_runtime': 70.9269,
 'eval_samples_per_second': 11.025,
 'eval_steps_per_second': 1.382,
 'epoch': 3.0}